# Using FastText via Gensim

This tutorial is about using [fastText](https://github.com/facebookresearch/fastText) model in Gensim. There are two ways you can use fastText in Gensim - Gensim's native implementation of fastText and Gensim wrapper for fastText's original C++ code. Here, we'll learn to work with fastText library for training word-embedding models, saving & loading them and performing similarity operations & vector lookups analogous to Word2Vec.

## When to use FastText?
The main principle behind fastText is that the morphological structure of a word carries important information about the meaning of the word, which is not taken into account by traditional word embeddings, which train a unique word embedding for every individual word. This is especially significant for morphologically rich languages (German, Turkish) in which a single word can have a large number of morphological forms, each of which might occur rarely, thus making it hard to train good word embeddings.  
fastText attempts to solve this by treating each word as the aggregation of its subwords. For the sake of simplicity and language-independence, subwords are taken to be the character ngrams of the word. The vector for a word is simply taken to be the sum of all vectors of its component char-ngrams.  
According to a detailed comparison of Word2Vec and FastText in [this notebook](Word2Vec_FastText_Comparison.ipynb), fastText does significantly better on syntactic tasks as compared to the original Word2Vec, especially when the size of the training corpus is small. Word2Vec slightly outperforms FastText on semantic tasks though. The differences grow smaller as the size of training corpus increases.
Training time for fastText is significantly higher than the Gensim version of Word2Vec (`15min 42s` vs `6min 42s` on text8, 17 mil tokens, 5 epochs, and a vector size of 100).  
fastText can be used to obtain vectors for out-of-vocabulary (OOV) words, by summing up vectors for its component char-ngrams, provided at least one of the char-ngrams was present in the training data.

## Training models

For the following examples, we'll use the Lee Corpus (which you already have if you've installed gensim) for training our model.

For using the wrapper for fastText, you need to have fastText setup locally to be able to train models. See [installation instructions for fastText](https://github.com/facebookresearch/fastText/#requirements) if you don't have fastText installed already.

### Using Gensim's implementation of fastText

In [1]:
from gensim.models.fasttext import FastText as FT_gensim
from gensim.test.utils import datapath

# Set file names for train and test data
corpus_file = datapath('lee_background.cor')

model_gensim = FT_gensim(size=100)

# build the vocabulary
model_gensim.build_vocab(corpus_file=corpus_file)

# train the model
model_gensim.train(
    corpus_file=corpus_file, epochs=model_gensim.epochs,
    total_examples=model_gensim.corpus_count, total_words=model_gensim.corpus_total_words
)

print(model_gensim)

FastText(vocab=1762, size=100, alpha=0.025)


### Using wrapper for fastText's C++ code

In [2]:
from gensim.models.wrappers.fasttext import FastText as FT_wrapper

# Set FastText home to the path to the FastText executable
ft_home = '/home/misha/src/fastText-0.1.0/fasttext'

# train the model
model_wrapper = FT_wrapper.train(ft_home, corpus_file)

print(model_wrapper)

FastText(vocab=1763, size=100, alpha=0.025)


### Training hyperparameters

Hyperparameters for training the model follow the same pattern as Word2Vec. FastText supports the following parameters from the original word2vec - 
     - model: Training architecture. Allowed values: `cbow`, `skipgram` (Default `cbow`)
     - size: Size of embeddings to be learnt (Default 100)
     - alpha: Initial learning rate (Default 0.025)
     - window: Context window size (Default 5)
     - min_count: Ignore words with number of occurrences below this (Default 5)
     - loss: Training objective. Allowed values: `ns`, `hs`, `softmax` (Default `ns`)
     - sample: Threshold for downsampling higher-frequency words (Default 0.001)
     - negative: Number of negative words to sample, for `ns` (Default 5)
     - iter: Number of epochs (Default 5)
     - sorted_vocab: Sort vocab by descending frequency (Default 1)
     - threads: Number of threads to use (Default 12)
     
In addition, FastText has three additional parameters - 
    - min_n: min length of char ngrams (Default 3)
    - max_n: max length of char ngrams (Default 6)
    - bucket: number of buckets used for hashing ngrams (Default 2000000)
Parameters `min_n` and `max_n` control the lengths of character ngrams that each word is broken down into while training and looking up embeddings. If `max_n` is set to 0, or to be lesser than `min_n`, no character ngrams are used, and the model effectively reduces to Word2Vec.

To bound the memory requirements of the model being trained, a hashing function is used that maps ngrams to integers in 1 to K. For hashing these character sequences, the [Fowler-Noll-Vo hashing function](http://www.isthe.com/chongo/tech/comp/fnv) (FNV-1a variant) is employed.

**Note:** As in the case of Word2Vec, you can continue to train your model while using Gensim's native implementation of fastText.

## Saving/loading models

Models can be saved and loaded via the `load` and `save` methods.

In [3]:
# saving a model trained via Gensim's fastText implementation
model_gensim.save('saved_model_gensim')
loaded_model = FT_gensim.load('saved_model_gensim')
print(loaded_model)

# saving a model trained via fastText wrapper
model_wrapper.save('saved_model_wrapper')
loaded_model = FT_wrapper.load('saved_model_wrapper')
print(loaded_model)

FastText(vocab=1762, size=100, alpha=0.025)
FastText(vocab=1763, size=100, alpha=0.025)


The `save_word2vec_method` causes the vectors for ngrams to be lost. As a result, a model loaded in this way will behave as a regular word2vec model.  


## Word vector lookup
**Note:** Operations like word vector lookups and similarity queries can be performed in exactly the same manner for both the implementations of fastText so they have been demonstrated using only the fastText wrapper here.

FastText models support vector lookups for out-of-vocabulary words by summing up character ngrams belonging to the word.

In [4]:
print('night' in model_wrapper.wv.vocab)
print('nights' in model_wrapper.wv.vocab)
print(model_wrapper['night'])
print(model_wrapper['nights'])

True
False
[ 0.8314139   0.61584824 -0.22241311  0.07523467  0.5152522   0.07724247
 -0.13744526  0.05606242 -0.09502476  0.45655364  0.51096547 -0.13521144
 -0.7620124  -0.4685431  -0.15228595 -0.03442579  0.20600994 -0.5080321
 -0.6443741   0.605772   -0.30647403  0.41962707  0.06037483 -0.40195057
 -0.11246474 -0.59829116 -0.32052496 -0.48515126  0.2997839  -0.20067295
 -0.20996568  0.12522118 -0.0364657   0.62870216  0.5781912  -0.00992062
  0.51955134 -0.10997857  0.16197589  0.27111182 -0.06318171 -0.24831475
  0.09808698 -0.37751442 -0.13298641 -0.15047912 -0.01828656 -0.6400881
  0.28488973 -0.14948265  0.18325825  0.6458386  -0.00953633  0.13587084
 -0.1961209  -0.42555386 -0.19528134  0.52414805 -0.30868796 -0.5202228
 -0.10896837  0.06696089  0.44607309  0.37719652  0.08233636  0.24584875
 -0.80979943 -0.30543917 -0.15849951  0.16166946 -0.36826986 -0.00906481
 -0.14814071 -0.25263855 -0.41303173 -0.48292273 -0.05554645 -0.00310395
  0.21415223 -0.27768075  0.7148276   1.336

The word vector lookup operation only works if at least one of the component character ngrams is present in the training corpus. For example -

In [5]:
# Raises a KeyError since none of the character ngrams of the word `axe` are present in the training data
try:
    model_wrapper['axe']
except KeyError:
    #
    # trap the error here so it does not interfere
    # with the execution of the cells below
    #
    pass
else:
    assert False, 'the above code should have raised a KeyError'

The `in` operation works slightly differently from the original word2vec. It tests whether a vector for the given word exists or not, not whether the word is present in the word vocabulary. To test whether a word is present in the training word vocabulary -

In [6]:
# Tests if word present in vocab
print("word" in model_wrapper.wv.vocab)
# Tests if vector present for word
print("word" in model_wrapper)

False
True


## Similarity operations

Similarity operations work the same way as word2vec. **Out-of-vocabulary words can also be used, provided they have at least one character ngram present in the training data.**

In [7]:
print("nights" in model_wrapper.wv.vocab)
print("night" in model_wrapper.wv.vocab)
model_wrapper.similarity("night", "nights")

False
True


0.99999416

Syntactically similar words generally have high similarity in fastText models, since a large number of the component char-ngrams will be the same. As a result, fastText generally does better at syntactic tasks than Word2Vec. A detailed comparison is provided [here](Word2Vec_FastText_Comparison.ipynb).

### Other similarity operations

In [8]:
# The example training corpus is a toy corpus, results are not expected to be good, for proof-of-concept only
model_wrapper.most_similar("nights")

[('night', 0.9999646544456482),
 ('flights', 0.9999643564224243),
 ('rights', 0.999963641166687),
 ('night.', 0.9999594688415527),
 ('quarter', 0.9999569654464722),
 ('night,', 0.9999566078186035),
 ('hearing', 0.9999553561210632),
 ('better', 0.9999548196792603),
 ('eight', 0.9999544620513916),
 ('during', 0.999954342842102)]

In [9]:
model_wrapper.n_similarity(['sushi', 'shop'], ['japanese', 'restaurant'])

0.9999701

In [10]:
model_wrapper.doesnt_match("breakfast cereal dinner lunch".split())

'cereal'

In [11]:
model_wrapper.most_similar(positive=['baghdad', 'england'], negative=['london'])

[('suicide', 0.9997773170471191),
 ('decide', 0.9997694492340088),
 ('side', 0.9997690916061401),
 ('Minister', 0.9997668266296387),
 ('inside', 0.9997666478157043),
 ('Minister,', 0.99976646900177),
 ('ministers', 0.9997649192810059),
 ('Alliance', 0.9997645616531372),
 ('best', 0.9997645020484924),
 ('bombers', 0.9997643232345581)]

In [12]:
model_wrapper.accuracy(questions=datapath('questions-words.txt'))

[{'section': 'capital-common-countries', 'correct': [], 'incorrect': []},
 {'section': 'capital-world', 'correct': [], 'incorrect': []},
 {'section': 'currency', 'correct': [], 'incorrect': []},
 {'section': 'city-in-state', 'correct': [], 'incorrect': []},
 {'section': 'family',
  'correct': [],
  'incorrect': [('HE', 'SHE', 'HIS', 'HER'), ('HIS', 'HER', 'HE', 'SHE')]},
 {'section': 'gram1-adjective-to-adverb', 'correct': [], 'incorrect': []},
 {'section': 'gram2-opposite', 'correct': [], 'incorrect': []},
 {'section': 'gram3-comparative',
  'correct': [('GREAT', 'GREATER', 'LOW', 'LOWER'),
   ('LONG', 'LONGER', 'LOW', 'LOWER'),
   ('LOW', 'LOWER', 'GREAT', 'GREATER')],
  'incorrect': [('GOOD', 'BETTER', 'GREAT', 'GREATER'),
   ('GOOD', 'BETTER', 'LONG', 'LONGER'),
   ('GOOD', 'BETTER', 'LOW', 'LOWER'),
   ('GREAT', 'GREATER', 'LONG', 'LONGER'),
   ('GREAT', 'GREATER', 'GOOD', 'BETTER'),
   ('LONG', 'LONGER', 'GOOD', 'BETTER'),
   ('LONG', 'LONGER', 'GREAT', 'GREATER'),
   ('LOW', 'LO

In [13]:
# Word Movers distance
sentence_obama = 'Obama speaks to the media in Illinois'.lower().split()
sentence_president = 'The president greets the press in Chicago'.lower().split()

# Remove their stopwords.
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
sentence_obama = [w for w in sentence_obama if w not in stopwords]
sentence_president = [w for w in sentence_president if w not in stopwords]

# Compute WMD.
distance = model_wrapper.wmdistance(sentence_obama, sentence_president)
distance

1.1245153746934533